In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

 
while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
 
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)
 
  model_dir = pathlib.Path(model_dir)/"saved_model"
 
  model = tf.saved_model.load(str(model_dir))
 
  return model
 
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

model_name = 'ssdlite_mobilenet_v2_coco_2018_05_09'
detection_model = load_model(model_name)

    

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
detection_model.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(input_tensor) at 0x2959838AB20>}))

In [7]:
concrete_func = detection_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0]
concrete_func.outputs

[<tf.Tensor 'Identity:0' shape=(1, 100) dtype=float32>,
 <tf.Tensor 'Identity_1:0' shape=(1, 100, 4) dtype=float32>,
 <tf.Tensor 'Identity_2:0' shape=(1, 100) dtype=float32>,
 <tf.Tensor 'Identity_3:0' shape=(1, 100, 91) dtype=float32>,
 <tf.Tensor 'Identity_4:0' shape=(1, 100) dtype=float32>,
 <tf.Tensor 'Identity_5:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'Identity_6:0' shape=(1, 51150, 4) dtype=float32>,
 <tf.Tensor 'Identity_7:0' shape=(1, 51150, 91) dtype=float32>]

In [8]:
import playsound as p
import threading

#to generate the corresponding audio label of the detected object

#def audioLabel(index):
#    i=0
#    path = "C:/Users/Raihan/computerVision/models/research/object_detection/audio_feedback/cocotext.txt"
#    with open(path) as f:
#        for line in f:
#            line=f.read().splitlines()
#            for items in line:
#                content = items
#                i+=1
#                if i == index:
#                    break
#    return(content)

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
  print(input_tensor)
  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
  print(output_dict)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  # num_detections = output_0, detection_scores = output_1, detection_classes = output_2, detection_boxes = output_3
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  #output_dict['detection_boxes'] = output_dict['output_3'] 
  #output_dict['detection_scores'] = output_dict['output_1']
    
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
   
  return output_dict
#delay = 0

def show_inference(model, frame):
  #dim = (320,320)
  #frame = cv2.resize(frame,dim)
  #global delay
  #delay+=1
  #take the frame from webcam feed and convert that to array
  image_np = np.array(frame)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      #instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=5)
    
      
    
#  if delay==20:
#        delay=0
#        filename = ''
#        base = 'C:/Users/Raihan/computerVision/models/research/object_detection/audio_feedback/audio/'
#        ext = '.mp3'
#        for i in output_dict['detection_classes']:
#            filename = audioLabel(i)
#            file = base+filename+ext
#            print(file)
#            threading.Thread(target=p.playsound, args=(file,), daemon=True).start()

         
  return(image_np)

In [9]:
import cv2
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re,frame = video_capture.read()
    Imagenp=show_inference(detection_model, frame)
    cv2.imshow('object detection', cv2.resize(Imagenp, (800,600)))
   
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

tf.Tensor(
[[[[225 255 213]
   [219 255 210]
   [214 255 207]
   ...
   [255 255 234]
   [255 255 235]
   [255 255 235]]

  [[226 255 210]
   [221 255 209]
   [217 255 207]
   ...
   [255 255 235]
   [255 255 237]
   [255 255 237]]

  [[223 255 198]
   [221 255 200]
   [219 255 201]
   ...
   [255 255 233]
   [255 255 235]
   [255 255 235]]

  ...

  [[ 82 125 125]
   [ 95 123 126]
   [110 123 128]
   ...
   [167 188 195]
   [195 202 201]
   [207 212 211]]

  [[ 85 128 131]
   [ 93 125 131]
   [103 122 129]
   ...
   [181 198 197]
   [214 218 209]
   [224 227 216]]

  [[ 94 127 131]
   [ 98 125 131]
   [104 123 130]
   ...
   [207 214 209]
   [227 228 216]
   [233 233 220]]]], shape=(1, 480, 640, 3), dtype=uint8)
{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[51072., 36467., 47499., 37417., 47273., 51072., 51073., 50657.,
        51072., 47507., 50847., 50907., 50807., 50906., 51072., 51062.,
        36467., 36953., 47508., 50645., 51072., 37433.

tf.Tensor(
[[[[200 237 195]
   [198 236 190]
   [196 235 186]
   ...
   [248 255 233]
   [248 255 233]
   [248 255 233]]

  [[201 237 190]
   [199 236 184]
   [199 237 181]
   ...
   [248 255 233]
   [246 255 231]
   [246 255 231]]

  [[199 235 184]
   [198 236 183]
   [196 236 183]
   ...
   [249 255 231]
   [248 255 229]
   [248 255 229]]

  ...

  [[ 75  92 108]
   [ 78  92 112]
   [ 81  92 116]
   ...
   [153 163 171]
   [160 165 171]
   [165 170 176]]

  [[ 74  89 111]
   [ 76  90 115]
   [ 79  91 120]
   ...
   [168 170 178]
   [183 182 189]
   [191 189 196]]

  [[ 72  87 115]
   [ 75  90 119]
   [ 74  90 118]
   ...
   [177 178 183]
   [200 198 196]
   [205 203 200]]]], shape=(1, 480, 640, 3), dtype=uint8)
{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[51102., 36947., 47195., 35993., 50717., 46949., 51072., 50118.,
        46955., 51072., 37432., 51066., 37433., 50711., 50644., 50238.,
        36937., 35993., 47425., 47195., 50847., 43278.

tf.Tensor(
[[[[195 236 196]
   [194 239 197]
   [192 240 197]
   ...
   [247 255 228]
   [248 255 229]
   [248 255 230]]

  [[193 236 191]
   [192 239 192]
   [192 242 194]
   ...
   [252 255 229]
   [250 255 229]
   [250 255 230]]

  [[190 235 188]
   [192 239 190]
   [194 242 192]
   ...
   [253 255 229]
   [252 255 229]
   [252 255 230]]

  ...

  [[ 66  93 106]
   [ 62  88 102]
   [ 63  87 101]
   ...
   [130 155 177]
   [135 160 181]
   [136 161 181]]

  [[ 67  91 107]
   [ 64  88 103]
   [ 66  88 102]
   ...
   [134 155 177]
   [149 168 187]
   [160 179 197]]

  [[ 72  92 113]
   [ 69  90 110]
   [ 67  88 107]
   ...
   [146 159 176]
   [175 184 195]
   [189 197 207]]]], shape=(1, 480, 640, 3), dtype=uint8)
{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[51072., 50725., 36947., 50725., 37808., 51076., 50723., 46679.,
        37432., 48643., 50723., 51070., 50725., 37433., 37814., 47748.,
        47518., 35993., 35033., 51066., 46204., 50725.

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[50725., 36467., 37809., 37815., 36467., 51074., 51073., 46211.,
        51066., 37433., 50711., 51072., 37433., 35993., 51073., 50705.,
        50842., 35033., 51073., 51076., 51076., 18875., 37809., 38282.,
        37814., 50777., 50912., 47509., 50119., 51076., 47273., 36473.,
        43331., 37815., 37809., 50235., 37815., 51073., 37815., 50906.,
        47936., 50244., 37809., 47503., 51072., 47702., 37809., 51076.,
        50913., 37333., 51072., 51073., 42617., 50107., 37815., 50252.,
        50807., 47508., 50670., 51076., 35508., 37433., 35993., 50644.,
        36939., 51130., 37815., 47697., 50133., 36953., 37433., 46445.,
        38289., 51066., 50116., 50777., 37814., 18881., 47273., 38282.,
        37815., 50247.,   140., 47461., 48611., 37815., 50704., 50258.,
        36467., 50584., 50905., 51076., 37794., 38282., 50913., 14404.,
        36947., 46961., 50644.,   140.]], dtype=float32)>

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[50725., 49523., 50800., 36947., 49763., 49282., 49403., 50722.,
        49763., 49763., 51066., 50800., 49763., 49643., 49763., 49996.,
        37820., 51076., 37432., 50860., 49999., 49763., 51076., 50861.,
        49996., 49883., 36472., 51042., 35993., 35993., 36937., 51042.,
        50807., 37820., 49763., 51072., 49763., 50801., 51067., 51069.,
        49997., 50800., 49996., 50119., 50812., 50730., 50800., 51133.,
        49996., 36953., 48889., 50860., 51073., 49997., 47743., 50801.,
        50644., 37803., 37820., 47508., 50644., 49763., 50913., 50776.,
        50116., 49769., 49763., 37433., 51072., 46445., 49277.,   140.,
        51076., 50860., 51072., 50807., 37820., 50860., 47508., 50644.,
        47739., 49883.,   140., 50860., 35033., 51043., 50850., 37335.,
        50860., 37433., 49763., 49158., 37833., 49997., 51066., 50651.,
        49763., 51067., 40663., 36472.]], dtype=float32)>

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[50725., 49397., 50800., 35993., 49757., 49757., 37820., 49757.,
        51079., 46685., 50728., 51076., 49757., 37432., 49637., 47508.,
        50807., 37820., 50794., 51042., 49757., 49277., 51073., 51072.,
        50117., 46655., 50800., 49277., 51076., 51076., 50872., 37820.,
        49877., 46655., 35993., 37433., 51067., 35993., 51066., 51072.,
        50089., 46205., 49877., 51042., 50860., 49997., 50859., 37820.,
        51139., 50928., 49999., 38294., 50801., 51075., 50092., 49883.,
        50119., 50800., 36953., 35032., 37820., 51066., 50644., 36953.,
        50644., 46931., 50800., 37820., 47508., 36953., 50644., 50800.,
        36953., 47509., 51076., 50584., 37820., 51072., 51072., 50716.,
        49036., 49997., 51127., 37820., 46450., 51076., 37820., 50123.,
        46660., 37820., 50644., 47508., 50123., 47141., 50651., 50920.,
        47508., 47131., 49157., 51073.]], dtype=float32)>

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[50725., 49355., 50782., 36947., 37809., 51074., 50079., 50783.,
        50842., 50783., 49835., 35993., 51066., 50776., 49963., 37432.,
        49355., 50064., 50898., 50079., 50776., 36953., 50837., 51075.,
        50662., 50782., 50665., 50837., 51073., 50898., 37433., 50902.,
        51073., 51072., 51066., 49715., 37433., 49961., 50842., 37815.,
        50777., 50185., 50644., 49715., 50584., 47696., 51072., 51076.,
        49013., 50902., 50901., 50854., 49835., 50069., 35993., 51060.,
        37809., 49835., 50901., 51067., 48611., 35993., 50075., 36937.,
        50712., 49715., 47278., 50783., 49355., 50904., 50898., 50902.,
        38276., 49110., 50898., 50710., 47278., 50898., 50894., 49835.,
        49835., 50843., 47691., 49595., 50898., 50679., 37432., 50085.,
        50838., 49841., 50898., 50782., 49121., 50898., 50902., 50837.,
        50846., 50783., 37797., 49835.]], dtype=float32)>

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[51072., 49775., 50807., 50752., 51072., 50806., 50872., 36947.,
        50866., 51078., 50806., 50665., 36463., 50258., 51067., 51044.,
        51067., 51100., 50711., 50806., 51067., 50806., 36463., 49414.,
        50717., 43282., 51043., 50693., 50665., 37417., 50806., 50866.,
        50131., 43042., 50919., 51067., 47503., 51067., 50806., 49291.,
        50797., 50922., 50644., 49170., 50806., 50806., 36463., 50693.,
        49553., 50615., 51044., 50924., 50129., 50806., 47195., 51139.,
        50866., 51072.,   140., 46715., 48851., 50859., 51100., 46715.,
        50866., 44123., 50866., 47272., 50863., 50584., 43282., 36463.,
        50665., 43042., 36463., 50810., 50782., 50747., 50806., 50651.,
        36953., 49288., 49427., 36463., 50866., 51084., 46715., 50905.,
        50866., 43042., 50152., 48947., 49312., 47273., 47512., 49073.,
        51078., 51078., 50907., 47750.]], dtype=float32)>

{'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[51072., 49553., 50818., 49673., 46540., 50878., 49312., 47488.,
        49673., 46781., 51066., 50711., 43282., 51076., 50033., 50152.,
        51066., 46294., 49552., 51076., 46540., 51067., 50807., 50717.,
        51104., 51075., 46060., 47021., 50644., 50878., 51076., 50782.,
        50866., 48731., 47493., 50878., 49673., 46540., 51043., 51072.,
        37373., 33079., 49312., 50907., 51044., 50152., 50879., 51044.,
        20326.,   146., 46961., 20332., 49913., 50671., 49913., 49673.,
        50906., 49793., 50615., 46540., 50152., 51076., 46301., 46061.,
        49673., 50878., 30679., 46541., 50674., 50680., 51131., 51139.,
        50818., 31643., 50912., 50878., 50776., 50724., 42803., 47207.,
        46955., 48869., 47251., 49312., 50033., 50773., 19852., 33559.,
        50644., 50033., 42887., 50776., 49913., 34037., 46313., 35927.,
        46300., 46294.,   134., 50800.]], dtype=float32)>